Revoir minimum WR distances : très long 

In [6]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from swiss_rounds_simulator.utils import simulate_tournament, simulate_n_tournaments, compare_settings


New LDC format places of interest :
* The 8 first teams qualify directly to round of 16
* From 9 to 24 : pseudo round of 32
* Interesting teams : i (9th) and y(25th)  

Format :
* 36 teams,
* 8 games,
* Hat determination of the first 2 games

LoL :
* 16 teams,
* Top 8 to Quarter finales,
* 6 games (not exactly, teams with 3 win qualifies, team with 3 loss gets eliminated)


In [12]:
simulate_tournament(16,6, delta_level = 'exponential', verbose=False)


,Id,Level,Strategy,Nb_win,Nb_games,Win_rate,R1_opponent,R1_result,R2_opponent,R2_result,R3_opponent,R3_result,R4_opponent,R4_result,R5_opponent,R5_result,R6_opponent,R6_result
d,4,0.125,[],6,6,1.0,l,Win,h,Win,b,Win,a,Win,c,Win,e,Win
b,2,0.5,[],4,6,0.666667,j,Win,f,Win,d,Loss,g,Win,a,Win,c,Loss
e,5,0.0625,[],4,6,0.666667,m,Win,a,Loss,i,Win,l,Win,f,Win,d,Loss
a,1,1.0,[],4,6,0.666667,i,Win,e,Win,c,Win,d,Loss,b,Loss,g,Win
c,3,0.25,[],4,6,0.666667,k,Win,g,Win,a,Loss,f,Win,d,Loss,b,Win
h,8,0.007812,[],4,6,0.666667,p,Win,d,Loss,l,Loss,k,Win,j,Win,n,Win
g,7,0.015625,[],3,6,0.5,o,Win,c,Loss,k,Win,b,Loss,i,Win,a,Loss
f,6,0.03125,[],3,6,0.5,n,Win,b,Loss,j,Win,c,Loss,e,Loss,l,Win
i,9,0.003906,[],3,6,0.5,a,Loss,m,Win,e,Loss,n,Win,g,Loss,o,Win
j,10,0.001953,[],3,6,0.5,b,Loss,n,Win,f,Loss,o,Win,h,Loss,m,Win


In [14]:
compare_settings(n_tournaments=100, 
                 n_teams=8, 
                 n_rounds = 4, 
                 delta_level ='linear',
                 strategies={'d':[1]}, 
                 probabilistic=True, 
                 deterministic=True)


Probabilistic resolution


AttributeError: 'ScoreBracket' object has no attribute '_m1'